<a href="https://colab.research.google.com/github/mahuthu/SQL_Slim-_model/blob/main/Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git lfs install

Git LFS initialized.


In [3]:
!git clone https://huggingface.co/llmware/slim-sql-1b-v0


Cloning into 'slim-sql-1b-v0'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 58 (delta 25), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (58/58), 495.00 KiB | 1.43 MiB/s, done.


In [ ]:
cd slim-sql-1b-v0


/content/slim-sql-1b-v0


In [ ]:
ls

config.json                     README.md                    tokenizer_config.json
generation_test_sql_slim_hf.py  special_tokens_map.json      tokenizer.json
pytorch_model.bin               sql_test_100_simple_s.jsonl  tokenizer.model


In [4]:
!pip install transformers


In [5]:
!pip install jsonlines


In [6]:
!git config --global credential.helper store

In [7]:
!pip install huggingface_hub

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
import sqlite3
import jsonlines
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [10]:
from google.colab import files
uploaded = files.upload()

Saving special_tokens_map.json to special_tokens_map.json


In [17]:


# Load the pre-trained GPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("llmware/slim-sql-1b-v0", special_token_map_file="special_token_map.json")


model = AutoModelForCausalLM.from_pretrained("llmware/slim-sql-1b-v0")


In [14]:
from google.colab import files
uploaded = files.upload()

Saving sql1.json to sql1.json


In [18]:
def execute_sql_query(sql_query):
    # Connect to a SQLite in-memory database
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()

    # Create a dummy table and insert some data
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')
    cursor.execute('''INSERT INTO users (name, age) VALUES ('John', 30)''')
    cursor.execute('''INSERT INTO users (name, age) VALUES ('Alice', 25)''')
    cursor.execute('''INSERT INTO users (name, age) VALUES ('Bob', 35)''')
    conn.commit()

    # Execute the SQL query and retrieve the results
    cursor.execute(sql_query)
    result = cursor.fetchall()

    # Close the connection
    conn.close()

    return result

In [22]:
import json
import os
import re
import random

In [25]:
!pip install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.3/796.3 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 k

In [27]:
!pip install --upgrade urllib3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.27.96 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.0 which is incompatible.


In [29]:
import time
from werkzeug.utils import secure_filename
import ast

from transformers import AutoModelForCausalLM, AutoTokenizer

from llmware.models import ModelCatalog
from llmware.prompts import Prompt

def model_test_run_general():

    t0 = time.time()

    model_name = "llmware/slim-sql-1b-v0"

    print("update: model_name - ", model_name)

    custom_hf_model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True)

    hf_tokenizer = AutoTokenizer.from_pretrained(model_name)

    #   now, we have 'imported' our own custom 'instruct' model into llmware
    model = ModelCatalog().load_hf_generative_model(custom_hf_model, hf_tokenizer, instruction_following=False,
                                                    prompt_wrapper="human_bot")

    model.temperature = 0.3
    # run direct inference on model
    print("\nupdate: Starting Generative Instruct Custom Fine-tuned Test")

    t1 = time.time()

    print("update: time loading model - ", t1 - t0)

ImportError: cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (/usr/local/lib/python3.10/dist-packages/urllib3/util/ssl_.py)

In [23]:
fp = ""
fn = "sql1.json"

opened_file = open(os.path.join(fp, fn), "r")


prompt_list = []

for i, rows in enumerate(opened_file):
    # print("update: ", i, rows)
    rows = json.loads(rows)
    new_entry = {"question": rows["question"],
                  "context": rows["context"]}

    prompt_list.append(new_entry)

random.shuffle(prompt_list)

total_response_output = []
perfect_match = 0

for i, entries in enumerate(prompt_list):
    prompt = entries["question"]
    context = re.sub("[\n\r]","", entries["context"])
    context = re.sub("\s+", " ", context)
    context = re.sub("\"", "", context)

    output = model.inference(prompt, add_context=context, add_prompt_engineering=True)

    print("\nupdate: model question - ", prompt)

    llm_response = re.sub("['\"]", "", output["llm_response"])
    answer = re.sub("['\"]", "", answer)

    print("update: model response - ", i, llm_response)

AttributeError: 'LlamaForCausalLM' object has no attribute 'inference'

In [21]:
data = []
with jsonlines.open('sql1.json') as reader:
    for obj in reader:
        data.append(obj)

# Iterate over each object in the JSON file
for obj in data:
    # Parse JSON objects
    context = obj['context']
    question = obj['question']

    # Generate SQL query using the GPT model
    input_text = f"{context} {question}"  # Properly format the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Ensure that the attention mask is created with all 1s (indicating all tokens are attended to)
    attention_mask = torch.ones_like(input_ids)

    # Generate output with attention mask, ensuring only one sequence is returned
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=100, num_return_sequences=1, early_stopping=True)
    generated_sql_query = tokenizer.decode(output[0])

    # Clean up the generated SQL query
    generated_sql_query = generated_sql_query.strip()  # Remove leading/trailing whitespace
    generated_sql_query = generated_sql_query.replace('<p>', '').replace('</p>', '')  # Remove HTML-like tags if present

    print("Generated SQL query:", generated_sql_query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL query: <s> The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL query: <s> The 'users' table contains information about users. Who is the youngest user?
What is the name of the youngest user?
The 'users' table contains information about users. Who is the oldest user?
What is the name of the oldest user?
The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the maximum age
Generated SQL query: <s> The 'users' table contains information about users. What is the total number of users?
What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. What is the total number of users?


In [ ]:

# data = []
# with jsonlines.open('sql1.json') as reader:
#     for obj in reader:
#         data.append(obj)



#     # Iterate over each object in the JSON file
# for obj in data:
#     # Parse JSON objects
#     context = obj['context']
#     question = obj['question']

#     # Generate SQL query using the GPT model
#     input_text = f"{context} {question}"  # Properly format the input text
#     input_ids = tokenizer.encode(input_text, return_tensors='pt')

#     # Ensure that the attention mask is created with all 1s (indicating all tokens are attended to)
#     attention_mask = torch.ones_like(input_ids)

#     # Generate output with attention mask, ensuring only one sequence is returned
#     output = model.generate(input_ids, attention_mask=attention_mask, max_length=100, num_return_sequences=1, early_stopping=True)
#     generated_sql_query = tokenizer.decode(output[0], skip_special_tokens=True)

#     # Clean up the generated SQL query
#     generated_sql_query = generated_sql_query.strip()  # Remove leading/trailing whitespace
#     generated_sql_query = generated_sql_query.replace('<p>', '').replace('</p>', '')  # Remove HTML-like tags if present

#     print("Generated SQL query:", generated_sql_query)
    # # Execute SQL query on an in-memory SQLite database and retrieve results
    # result = execute_sql_query(generated_sql_query)

    # # Print the generated SQL query and the result
    # print("Generated SQL query:", generated_sql_query)
    # print("Query result:", result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL query: The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL query: The 'users' table contains information about users. Who is the youngest user?
What is the name of the youngest user?
The 'users' table contains information about users. Who is the oldest user?
What is the name of the oldest user?
The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the maximum age
Generated SQL query: The 'users' table contains information about users. What is the total number of users?
What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. What is the total number of users?


In [13]:
prompts = [
    "The 'users' table contains information about users. What is the average age of users?",
    "The 'users' table contains information about users. Who is the youngest user?",
    "The 'users' table contains information about users. What is the total number of users?"
    # Add more prompts as needed
]

# Generate SQL queries for each prompt
for prompt in prompts:
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt', padding=True)

    # Generate output
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, early_stopping=True)
    generated_sql_query = tokenizer.decode(output[0], skip_special_tokens=True)

    # Clean up the generated SQL query
    generated_sql_query = generated_sql_query.strip()  # Remove leading/trailing whitespace
    generated_sql_query = generated_sql_query.replace('<p>', '').replace('</p>', '')  # Remove HTML-like tags if present

    print("Prompt:", prompt)
    print("Generated SQL query:", generated_sql_query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: The 'users' table contains information about users. What is the average age of users?
Generated SQL query: The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?
The 'users' table contains information about users. What is the average age of users?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: The 'users' table contains information about users. Who is the youngest user?
Generated SQL query: The 'users' table contains information about users. Who is the youngest user?
What is the name of the youngest user?
The 'users' table contains information about users. Who is the oldest user?
What is the name of the oldest user?
The 'users' table contains information about users. What is the average age of users?
What is the average age of users?
The 'users' table contains information about users. What is the maximum age
Prompt: The 'users' table contains information about users. What is the total number of users?
Generated SQL query: The 'users' table contains information about users. What is the total number of users?
What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. What is the total number of users?
The 'users' table contains information about users. Wha